# Spheres

Fivethirtyeight's Riddler asks "[what is the minimum number of spheres of sequential size (1,2,3, etc.) that can be divided into three equal-sized groups](https://fivethirtyeight.com/features/can-you-flip-the-magic-coin/)? By "equal size", the problem refers to weight, but since all the spheres in this case are made of the same material (gold, as it happens), this is equivalent to finding groups of equal volume. We can further simplify the problem: the volume of a sphere of radius $r$ is $\frac{4}{3}\pi r^3$; we can pull $\frac{4}{3}\pi$ out of each volume to make this equivalent to finding the smallest $n$ such that the first $n$ cubes can be partitioned into three groups of equal sum.

(After re-reading the problem, I suppose I should also note that the spheres have whole number **diameters**, not radii; fortunately, this does not alter the solution.)

I will use Julia to work this problem, in an attempt to learn more of the language. We start with a function for the sum of cubes:

In [1]:
function sumofcubes(n)
    return (n * (n+1) / 2)^2
end

sumofcubes (generic function with 1 method)

Initially, I had an algorithm to find **one** sublset summing to the correct total, but it led to wrong answers, because it failed to account for the fact that there are multiple combinations of cubes with the same sum (for example, $3^3 + 4^3 + 5^3 = 6^3$). In hindsight, it's a little surprising that such a greedy algorithm would find solutions at all. The new algorithm simply finds all possible subsets of a list that add to a given total.

In [2]:
# Algorithm:
# For each element in the list of cubes
#     1) put this element in a partition
#        decrement the total by that amount
#        try to find a subset of the remaining elements adding to the lower total
#     2) 

function allparts(list, total)
    if sum(list) < total
        return []
    end
    for idx in 1:length(list)
        elt = list[idx]
        if elt > total
            return [] # everything left is also too big
        end
        if elt == total # jackpot
            return Array{Int64,1}[[elt]] # have to cast this
        end
        result = []
        with = allparts(list[idx+1:end], total - elt)
        if !isempty(with)
            result = [append!([elt],list) for list in with]
        end
        without = allparts(list[idx+1:end], total)
        append!(result, without)
        return result
    end
end

allparts (generic function with 1 method)

We can eliminate certain values of $n$ as possibilities immediately. For one, if $n = 1 \left(\mod 3\right)$, then the sum of cubes won't be divisible by 3. Also, the sum of the cubes has to be larger than three times the largest cube; the smallest $n$ where this occurs is 10, but since this is already not a possibility, we can start with $n = 11$. 

In [3]:
n = 10 # I know I just said start at 11, but...
cubes = map(x -> x^3, 1:n)
result = []
while isempty(result) # loop until we find a solution
    n += 1 # ... adding one to n up front allows a quick continue
    append!(cubes, n^3)
    if n % 3 == 1
        continue
    end
    total = sumofcubes(n)/3
    parts = allparts(cubes, total)
    for firstpart in parts
        rest = filter(x -> isempty(intersect(x, firstpart)), parts)
        for secondpart in rest
            thirdpart = filter(x -> isempty(intersect(x, secondpart)), rest)
            if !isempty(thirdpart)
                result = [n,
                    firstpart,
                    secondpart,
                    pop!(thirdpart)]
            end
        end
    end
end
result 

4-element Array{Any,1}:
 23                                            
   [27, 216, 1000, 2197, 5832, 6859, 9261]     
   [8, 125, 729, 1331, 2744, 3375, 4913, 12167]
   [1, 64, 343, 512, 1728, 4096, 8000, 10648]  

Huzzah!
Not the most efficient or elegant algorithm ever, but we've got a solution.

## Extra Credit

Now, we are asked what the minimum number of spheres are that can be partitioned into 2, 4, 5, 6, or k sets. We've got the tools in place, we just need to change the loop a bit:

1. We need to pass the number of needed sets, instead of assuming it's 3.
2. We should also just check that the total needed for each partition is an integer.
3. A nested loop is no longer feasible, so we have to create a recursive function.

In [4]:
function findpartition(parts, k)
    if !isempty(parts)
        if k == 1
            return parts
        end
        for part in parts
            solution = findpartition(filter(x -> isempty(intersect(x, part)), parts), k-1)
            if !isempty(solution)
                return append!([part], solution)
            end
        end
    end
    return []
end

findpartition (generic function with 1 method)

In [5]:
function partitioncubes(k)
    if k == 1
        return [[1], [1]]
    end
    n = 4k - 2 # if n > 1, this is the lowest n for which k*n^3 is less than the sum of the fist n cubes
    cubes = map(x -> x^3, 1:n)
    result = []
    while true
        total = sumofcubes(n)/k
        if total % 1 == 0 # no fractional part
            parts = allparts(cubes, total)
            result = findpartition(parts, k)
            if !isempty(result)
                break # while loop
            end
        end # if total...
        n += 1
        append!(cubes, n^3)
    end # while loop
    return append!([[n]], result)
end

partitioncubes (generic function with 1 method)

Let's see how this goes:

In [6]:
partitioncubes(2)

3-element Array{Array{Int64,1},1}:
 [12]                           
 [1, 8, 64, 512, 729, 1728]     
 [27, 125, 216, 343, 1000, 1331]

In [7]:
partitioncubes(3) # better get 23

4-element Array{Array{Int64,1},1}:
 [23]                                        
 [1, 64, 343, 512, 1728, 4096, 8000, 10648]  
 [8, 125, 729, 1331, 2744, 3375, 4913, 12167]
 [27, 216, 1000, 2197, 5832, 6859, 9261]     

In [8]:
partitioncubes(4)

5-element Array{Array{Int64,1},1}:
 [24]                                          
 [1, 8, 27, 64, 2744, 5832, 13824]             
 [125, 216, 512, 1331, 2197, 3375, 4096, 10648]
 [343, 729, 9261, 12167]                       
 [1000, 1728, 4913, 6859, 8000]                

In [9]:
partitioncubes(5)

6-element Array{Array{Int64,1},1}:
 [24]                                    
 [1, 5832, 12167]                        
 [8, 64, 729, 3375, 13824]               
 [27, 125, 1728, 6859, 9261]             
 [216, 343, 1000, 1331, 2197, 4913, 8000]
 [512, 2744, 4096, 10648]                

In [10]:
partitioncubes(6)

7-element Array{Array{Int64,1},1}:
 [35]                                                  
 [1, 8, 27, 125, 1000, 2197, 8000, 12167, 15625, 27000]
 [64, 729, 2744, 9261, 13824, 17576, 21952]            
 [216, 3375, 29791, 32768]                             
 [343, 512, 4913, 6859, 10648, 42875]                  
 [1331, 5832, 19683, 39304]                            
 [1728, 4096, 24389, 35937]                            

Is this enough for a pattern? Probably not.
(Previously, there was more analysis here, but the error in my original algorithm makes it irrelevant)

In [11]:
partitioncubes(7)

8-element Array{Array{Int64,1},1}:
 [41]                                     
 [1, 8, 27, 64, 6859, 19683, 24389, 54872]
 [125, 1728, 5832, 15625, 35937, 46656]   
 [216, 2744, 4096, 17576, 21952, 59319]   
 [343, 10648, 12167, 13824, 68921]        
 [512, 1331, 2197, 29791, 32768, 39304]   
 [729, 4913, 9261, 27000, 64000]          
 [1000, 3375, 8000, 42875, 50653]         

This seems unhelpful. Perhaps there's an asymptotic result for k. Perhaps I'll see it someday.